In [ ]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/WELFake_Dataset.csv")
print(df.columns)
print(df.shape)
print(type(df.loc[7,"text"]))

text_iterator = df['text']
title_iterator = df['title']

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(72134, 4)
<class 'str'>


In [ ]:
def process_text(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise
            # t = t.split(" ")
            # drop empty string
            # t = list(filter(lambda x: x != "", t))
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text(text_iterator)
title_processed = process_text(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

72134
72134
now most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change loretta lynch aka eric holder in a skirt


In [ ]:
df_output = pd.DataFrame(columns=["title", "text", "class"])
df_output["title"] = title_processed
df_output["text"] = text_processed
df_output["class"] = df["label"]

In [ ]:
x_train = np.array(df_output["text"][:10000])
y_train = []
ls = []
for i in df_output["class"][:10000]:
  ls.append(i)
  y_train.append(ls)
  ls = []
y_train = np.array(y_train)


x_test = np.array(df_output["text"][10001:11001])
y_test = []
ls = []
for i in df_output["class"][10001:11001]:
  ls.append(i)
  y_test.append(ls)
  ls = []
y_test = np.array(y_test)

print(x_train[0:3])
print(y_test[0:10])

['no comment is expected from barack obama members of the fyf911 or fukyoflag and blacklivesmatter movements called for the lynching and hanging of white people and cops they encouraged others on a radio show tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in america one of the f yoflag organizers is called sunshine she has a radio blog show hosted from texas called sunshine s f ing opinion radio show a snapshot of her fyf911 lolatwhitefear twitter page at 9 53 p m shows that she was urging supporters to call now fyf911 tonight we continue to dismantle the illusion of white below is a snapshot twitter radio call invite fyf911the radio show aired at 10 00 p m eastern standard time during the show callers clearly call for lynching and killing of white people a 2 39 minute clip from the radio show can be heard here it was provided to breitbart texas by someone who would like to be referred to as hannibal he has already rece

In [ ]:
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

In [ ]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=1,max_df=2)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

In [ ]:
classifier.fit(x_train, y_train.ravel())
predicted = classifier.predict(x_test)
# predicted

num_correct = 0
for i in range(len(predicted)):
  if predicted[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

586
0.586


In [ ]:
# normal bag of words model without including length
classifier2 = Pipeline([
    ('vectorizer', CountVectorizer(min_df=1,max_df=2)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())])
classifier2.fit(x_train, y_train.ravel())

predicted2 = classifier2.predict(x_test)
num_correct = 0
for i in range(len(predicted2)):
  if predicted2[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

649
0.649
